In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import os

if "JAVA_HOME" not in os.environ:
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64/"


config = {
    "master": "local[7]"
}


spark = SparkSession.builder.appName("beton")

for p in config:
    spark = spark.config(p, config[p])

spark = spark.getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/02/01 14:55:40 WARN Utils: Your hostname, DESKTOP-FGNDTTH resolves to a loopback address: 127.0.1.1; using 172.30.187.250 instead (on interface eth0)
25/02/01 14:55:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/01 14:55:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/02/01 14:55:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
home_dir = "/mnt/e/Проекты/beton/"
complex_dir = home_dir + "Basmanny/"

In [5]:
import os
files = [complex_dir + f for f in os.listdir(complex_dir)]
files[:3]

['/mnt/e/Проекты/beton/Basmanny/885861.pdf',
 '/mnt/e/Проекты/beton/Basmanny/obj42009_pd77-002296 (1).pdf',
 '/mnt/e/Проекты/beton/Basmanny/obj42009_pd77-002296 (2).pdf']

In [6]:
import pymupdf # imports the pymupdf library
doc = pymupdf.open(files[0]) # open a document
text = ""
for page in doc: # iterate the document pages
  text += page.get_text() + "\n" # get plain text encoded as UTF-8

text = text.replace('\xad', '').replace('\u00ad', '').replace('\N{SOFT HYPHEN}', '')

In [7]:
strs = text.split("\n")
strs[:10]

['ПРОЕКТНАЯ ДЕКЛАРАЦИЯ',
 '№ 77-002296 от 08.07.2024',
 'ЖК MyPriority Basmanny, г.Москва, ЦАО, район Басманный, ул.Малая',
 'Почтовая вл.12, стр.2,3,4,5,7,911,вл.12, соор.6 с помещениями общественного',
 'назначения и подземной автостоянкой',
 'Дата первичного размещения: 30.07.2021',
 '01 О фирменном наименовании (наименовании) застройщика, месте нахождения застройщика, режиме',
 'его работы, номере телефона, адресе официального сайта застройщика и адресе электронной почты (при',
 'наличии), фамилии, имени, отчестве (если имеется) лица, исполняющего функции единоличного ис-',
 'полнительного органа застройщика, а также об индивидуализирующем застройщика коммерческом обоз-']

In [9]:
from pyspark.sql import Row

spark.createDataFrame([Row(s) for s in strs], "text: string").limit(10).toPandas()

,text
0,ПРОЕКТНАЯ ДЕКЛАРАЦИЯ
1,№ 77-002296 от 08.07.2024
2,"ЖК MyPriority Basmanny, г.Москва, ЦАО, район Б..."
3,"Почтовая вл.12, стр.2,3,4,5,7,911,вл.12, соор...."
4,назначения и подземной автостоянкой
5,Дата первичного размещения: 30.07.2021
6,01 О фирменном наименовании (наименовании) зас...
7,"его работы, номере телефона, адресе официально..."
8,"наличии), фамилии, имени, отчестве (если имеет..."
9,"полнительного органа застройщика, а также об и..."


In [33]:
dt = strs[1].split(" ")[-1]

import builtins

print("-".join(dt[::-1].split("\."))[::-1])
# yyyy-mm-dd

08.07.2024


In [42]:
dt = strs[1].split(" ")[-1]

import builtins

decl_dt = print("-".join(dt.split(".")[::-1]))
decl_dt
# yyyy-mm-dd

2024-07-08


In [43]:
spark.createDataFrame([Row(i, s) for i, s in enumerate(strs)], "rn: int, text: string").count()

25845

In [44]:
cached_text = spark.createDataFrame([Row(i, s) for i, s in enumerate(strs)], "rn: int, text: string").cache()

In [45]:
cached_text.limit(10).toPandas()

,rn,text
0,0,ПРОЕКТНАЯ ДЕКЛАРАЦИЯ
1,1,№ 77-002296 от 08.07.2024
2,2,"ЖК MyPriority Basmanny, г.Москва, ЦАО, район Б..."
3,3,"Почтовая вл.12, стр.2,3,4,5,7,911,вл.12, соор...."
4,4,назначения и подземной автостоянкой
5,5,Дата первичного размещения: 30.07.2021
6,6,01 О фирменном наименовании (наименовании) зас...
7,7,"его работы, номере телефона, адресе официально..."
8,8,"наличии), фамилии, имени, отчестве (если имеет..."
9,9,"полнительного органа застройщика, а также об и..."


In [55]:
rns = cached_text.where("text like '%19.7.1.1.1.1%' or text like '%19.7.2.1.1.1%' or text like '%19.7.3.1.1.1%'").toPandas().to_dict(orient="records")
rns = dict([(x["text"], x["rn"]) for x in rns])
rns

{'19.7.1.1.1.1': 24170, '19.7.2.1.1.1': 24231, '19.7.3.1.1.1': 24297}

In [ ]:
rns

In [56]:
rns["19.7.1.1.1.1"]

24170

In [60]:
cached_text.where(col("rn") == rns["19.7.1.1.1.1"] + 3).select("text").collect()[0][0]

'877'

In [62]:
flats = cached_text.where(col("rn") == rns["19.7.1.1.1.1"] + 3).select("text").withColumn("type", lit("flats"))
meters = cached_text.where(col("rn") == rns["19.7.2.1.1.1"] + 3).select("text").withColumn("type", lit("meters"))
price = cached_text.where(col("rn") == rns["19.7.3.1.1.1"] + 3).select("text").withColumn("type", lit("price"))

flats.union(meters).union(price).toPandas()

,text,type
0,877,flats
1,"49 275,01 м2",meters
2,18 315 298 000 руб.,price


In [65]:
spark.createDataFrame([Row(s) for s in files], "path: string").toPandas()[:4]

,path
0,/mnt/e/Проекты/beton/Basmanny/885861.pdf
1,/mnt/e/Проекты/beton/Basmanny/obj42009_pd77-00...
2,/mnt/e/Проекты/beton/Basmanny/obj42009_pd77-00...
3,/mnt/e/Проекты/beton/Basmanny/obj42009_pd77-00...


In [197]:
def parse(path):
    import pymupdf # imports the pymupdf library
    try:
        doc = pymupdf.open(path) # open a document
        text = ""
        
        for page in doc: # iterate the document pages
            text += page.get_text() + "\n" # get plain text encoded as UTF-8
        #    pass

        text = text.replace('\xad', '').replace('\u00ad', '').replace('\N{SOFT HYPHEN}', '')
        strs = text.split("\n")
    #
        dt = strs[1].split(" ")[-1]

        #decl_dt = path
#
        decl_dt = "-".join(dt.split(".")[::-1])
        return decl_dt, ""
        #return text #str(len(strs))
    except Exception as e:
        return None, str(e)

In [186]:
spark.createDataFrame([Row(s) for s in files], "path: string")\
.withColumn("result", udf(parse, "string")("path"))\
.limit(4)\
.toPandas()

,path,result
0,/mnt/e/Проекты/beton/Basmanny/885861.pdf,[Ljava.lang.Object;@5471617e
1,/mnt/e/Проекты/beton/Basmanny/obj42009_pd77-00...,[Ljava.lang.Object;@4f15b17e
2,/mnt/e/Проекты/beton/Basmanny/obj42009_pd77-00...,[Ljava.lang.Object;@444baed
3,/mnt/e/Проекты/beton/Basmanny/obj42009_pd77-00...,[Ljava.lang.Object;@3fa85817


In [198]:
def parse_pd(paths):
    for pdf in paths:
        dts = []
        for r in pdf.itertuples():
           res = parse(str(r[1]))
           dts.append(str(res[0]) + res[1])

        pdf["dt"] = dts #str(r[1])  
        #print(pdf)
        yield pdf

In [200]:
spark.createDataFrame([Row(s) for s in files], "path: string")\
.mapInPandas(
    parse_pd, schema="path: string, dt: string"
)\
.limit(4)\
.toPandas()

,path,dt
0,/mnt/e/Проекты/beton/Basmanny/885861.pdf,2024-07-08
1,/mnt/e/Проекты/beton/Basmanny/obj42009_pd77-00...,2024-09-17
2,/mnt/e/Проекты/beton/Basmanny/obj42009_pd77-00...,2024-09-30
3,/mnt/e/Проекты/beton/Basmanny/obj42009_pd77-00...,2024-10-08
